<a href="https://colab.research.google.com/github/smritiirani/Dcluttr-task/blob/main/SQL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install duckdb pandas


In [3]:
import duckdb
import pandas as pd
from google.colab import files


In [4]:
uploaded = files.upload()


Saving blinkit_city_map.csv to blinkit_city_map.csv
Saving blinkit_categories.csv to blinkit_categories.csv
Saving all_blinkit_category_scraping_stream.csv to all_blinkit_category_scraping_stream.csv


In [8]:
import pandas as pd

# Load CSV files
scraping_df = pd.read_csv("all_blinkit_category_scraping_stream.csv")
categories_df = pd.read_csv("blinkit_categories.csv")
city_map_df = pd.read_csv("blinkit_city_map.csv")




In [9]:
# Rename for consistency with the SQL logic
scraping_df = scraping_df.rename(columns={
    'created_at': 'scraped_at',
    'l1_category_id': 'category_id',
    'l2_category_id': 'subcategory_id',
    'selling_price': 'price'
})


In [10]:
import sqlite3

# Create in-memory SQLite DB
con = sqlite3.connect(":memory:")

# Store DataFrames as SQL tables
scraping_df.to_sql("all_blinkit_category_scraping_stream", con, index=False, if_exists="replace")
categories_df.to_sql("blinkit_categories", con, index=False, if_exists="replace")
city_map_df.to_sql("blinkit_city_map", con, index=False, if_exists="replace")


1023

In [11]:
query = '''
WITH inventory_movement AS (
    SELECT
        s.store_id,
        s.sku_id,
        s.category_id,
        s.subcategory_id,
        s.scraped_at,
        s.price,
        s.inventory,
        LAG(s.inventory) OVER (
            PARTITION BY s.store_id, s.sku_id
            ORDER BY s.scraped_at
        ) AS prev_inventory
    FROM all_blinkit_category_scraping_stream s
),
estimation AS (
    SELECT
        im.store_id,
        im.sku_id,
        im.category_id,
        im.subcategory_id,
        im.price,
        CASE
            WHEN im.prev_inventory IS NOT NULL AND im.inventory < im.prev_inventory
                THEN im.prev_inventory - im.inventory
            ELSE 0
        END AS est_qty_sold
    FROM inventory_movement im
),
joined_data AS (
    SELECT
        e.sku_id,
        c.category_name,
        c.subcategory_name,
        m.city,
        e.price,
        e.est_qty_sold
    FROM estimation e
    JOIN blinkit_categories c
        ON e.category_id = c.category_id AND e.subcategory_id = c.subcategory_id
    JOIN blinkit_city_map m
        ON e.store_id = m.store_id
)
SELECT
    city,
    category_name,
    subcategory_name,
    sku_id,
    ROUND(AVG(price), 2) AS avg_price,
    SUM(est_qty_sold) AS est_qty_sold
FROM joined_data
GROUP BY city, category_name, subcategory_name, sku_id
'''


In [13]:
# Check the columns in blinkit_categories
categories_df.columns


Index(['l1_category', 'l1_category_id', 'l2_category', 'l2_category_id'], dtype='object')

In [14]:
# Rename category columns
categories_df = categories_df.rename(columns={
    'l1_category': 'category_name',
    'l2_category': 'subcategory_name'
})


In [15]:
# Reload the renamed dataframe into SQLite
categories_df.to_sql("blinkit_categories", con, index=False, if_exists="replace")


2

In [19]:
con.execute("PRAGMA table_info(blinkit_city_map)").fetchall()


[(0, 'store_id', 'INTEGER', 0, None, 0), (1, 'city_name', 'TEXT', 0, None, 0)]

In [21]:
# Check the table structure
pd.read_sql_query("PRAGMA table_info(blinkit_categories);", con)


,cid,name,type,notnull,dflt_value,pk
0,0,category_name,TEXT,0,None,0
1,1,l1_category_id,INTEGER,0,None,0
2,2,subcategory_name,TEXT,0,None,0
3,3,l2_category_id,INTEGER,0,None,0


from matplotlib import pyplot as plt
_df_0['cid'].plot(kind='hist', bins=20, title='cid')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('name').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('type').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['cid']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'cid'}, axis=1)
              .sort_values('cid', ascending=True))
  xs = counted['cid']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('cid', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('name')):
  _plot_series(series, series_name, i)
  fig.legend(title='name', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('cid')
_ = plt.ylabel('count()')

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['type'].value_counts()
    for x_label, grp in _df_8.groupby('name')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('name')
_ = plt.ylabel('type')

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_9['name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_9, x='cid', y='name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_10['type'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_10, x='cid', y='type', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [23]:
con.execute("PRAGMA table_info(blinkit_city_map)").fetchall()


[(0, 'store_id', 'INTEGER', 0, None, 0), (1, 'city_name', 'TEXT', 0, None, 0)]

In [27]:
import pandas as pd

# Run the schema inspection query
schema_df = pd.read_sql_query("PRAGMA table_info(blinkit_categories);", con)

# Save it to a tab-separated text file
schema_df.to_csv("blinkit_categories_schema.txt", sep="\t", index=False)

# Download the file to your local machine
from google.colab import files
files.download("blinkit_categories_schema.txt")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>